In [1]:
from quantdev.data import Databank
from quantdev.backtest import *
from quantdev.analysis import *
# from quantdev.trade import PortfolioManager, Position, Portfolio

# from typing import Literal
# import pandas as pd
# import numpy as np

In [2]:
db = Databank()
# db.update_databank()

In [4]:
[col for col in (db.read_dataset('capital_formation').columns) if col.endswith('公告日')]

['增資公告日', '繳款憑證公告日', '現增股票公告日', '權利證書公告日', '員工紅利增資股公告日', '股票股利公告日', '增減資公告日']

In [9]:
db.read_dataset('capital_formation')#, columns=[col for col in (db.read_dataset('capital_formation').columns) if any([col.endswith('公告日'), col.endswith('日')])])

,stock_id,除權日,總股本(仟元),面額,幣別,總股數(仟股),可配股總股數(仟股),現金增資(仟股),盈餘增資(仟股),公積增資(仟股),...,配股(Y/N),增資失敗原因,私募(Y/N),其他-說明,董事會決議日,IPO(Y/N),私募定價日,資金用途,insert_time,增減資公告日
0,000116,2005-12-08,11572128.0,10.0,NTD,1157213.0,1057213.0,0.0,0.0,100000.0,...,Y,,N,,NaT,N,NaT,,2024-05-09 15:23:57.180990,NaT
1,000930,2020-09-25,8147743.0,10.0,NTD,814774.0,811174.0,3600.0,0.0,0.0,...,N,,Y,,2020-08-20,N,2020-09-22,充實營運資金，健全公司財務結構,2024-05-09 15:23:57.180990,NaT
2,000930,2020-10-14,6547743.0,10.0,NTD,654774.0,814774.0,0.0,0.0,0.0,...,N,,N,,2020-08-20,N,NaT,,2024-05-09 15:23:57.180990,NaT
3,000960,2005-04-15,12818000.0,10.0,NTD,1281800.0,1352103.0,0.0,0.0,0.0,...,N,,N,,NaT,N,NaT,,2024-05-09 15:23:57.180990,NaT
4,000960,2005-11-10,14098000.0,10.0,NTD,1409800.0,1281800.0,0.0,0.0,128000.0,...,Y,,N,,NaT,N,NaT,,2024-05-09 15:23:57.180990,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65176,9962,2010-08-19,784524.0,10.0,NTD,78452.0,73320.0,0.0,5132.0,0.0,...,Y,,N,,NaT,N,NaT,,2024-05-09 15:23:57.180990,NaT
65177,9962,2005-11-30,501244.0,10.0,NTD,50124.0,39468.0,0.0,10656.0,0.0,...,Y,,N,,NaT,N,NaT,,2024-05-09 15:23:57.180990,NaT
65178,9962,2011-07-12,902203.0,10.0,NTD,90220.0,78452.0,0.0,11768.0,0.0,...,Y,,N,,NaT,N,NaT,,2024-05-09 15:23:57.180990,NaT
65179,9962,2007-07-10,733200.0,10.0,NTD,73320.0,56400.0,16920.0,0.0,0.0,...,N,,N,,NaT,N,NaT,充實營運資金,2024-05-09 15:23:57.180990,NaT


In [7]:
Databank().read_dataset('mkt_calendar', columns=['date'], filters=[('休市原因中文說明(人工建置)','=','')]).rename(columns={'date':'t_date'}).dtypes

t_date    datetime64[ms]
dtype: object

In [4]:
pd.DataFrame(pd.date_range(start='2005-01-01', end='2025-03-20', freq='YS'), columns=['r_date']).dtypes

r_date    datetime64[ns]
dtype: object

#### data

In [10]:
db = Databank()
# db.update_tej_dataset('stock_trading_data')

In [13]:
import tejapi
tejapi.ApiConfig.api_key = '2p6Pkl4B86cEaEgv8Bavt3HZxuhNRU'

# Get stock trading data

In [19]:
test = tejapi.get(
    datatable_code = 'TWN/APISALE',
    mdate={'gte': '2005-01-01', 'lte': '2025-03-20'}, 
    chinese_column_name=True,
    paginate=True, 
)

In [20]:
test = tejapi.fastget(
    datatable_code = 'TWN/APISALE',
    mdate={'gte': '2005-01-01', 'lte': '2025-03-20'}, 
    chinese_column_name=True,
    paginate=True, 
)

In [12]:
df = db.read_dataset('monthly_rev')
df

,date,stock_id,release_date,單月營收(千元),去年單月營收(千元),單月營收成長率％,單月營收與上月比％,累計營收(千元),去年累計營收(千元),累計營收成長率％,...,近3月累計營收成長率％,近3月累計營收與上月比％,近3月累計營收變動率％,流通在外股數(千股),單月每股營收(元),累計每股營收(元),近12月每股營收(元),近 3月每股營收(元),t_date,insert_time
0,2005-01-01,1101,2005-02-14,NaN,NaN,35.65,5.50,NaN,NaN,35.65,...,21.59,5.97,18.52,NaN,NaN,NaN,NaN,NaN,2005-02-15,2024-12-08 13:28:48.466555
1,2005-01-01,1102,2005-02-14,NaN,NaN,43.53,9.76,NaN,NaN,43.53,...,23.07,8.60,18.38,NaN,NaN,NaN,NaN,NaN,2005-02-15,2024-12-08 13:28:48.466555
2,2005-01-01,1103,2005-02-14,NaN,NaN,56.40,4.90,NaN,NaN,56.40,...,27.72,3.24,18.85,NaN,NaN,NaN,NaN,NaN,2005-02-15,2024-12-08 13:28:48.466555
3,2005-01-01,1104,2005-02-14,NaN,NaN,49.11,-2.97,NaN,NaN,49.11,...,19.13,1.99,16.27,NaN,NaN,NaN,NaN,NaN,2005-02-15,2024-12-08 13:28:48.466555
4,2005-01-01,1108,2005-02-14,NaN,NaN,42.66,8.48,NaN,NaN,42.66,...,20.18,6.28,30.34,NaN,NaN,NaN,NaN,NaN,2005-02-15,2024-12-08 13:28:48.466555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361835,2025-02-01,9951,2025-03-05,455345.0,368284.0,23.64,-10.70,965275.0,814713.0,18.48,...,20.59,-3.65,2.36,74900.0,6.08,12.89,75.70,20.43,2025-03-06,2025-03-10 21:11:43.846563
361836,2025-02-01,9955,2025-03-07,69106.0,76868.0,-10.10,-15.46,150850.0,201392.0,-25.10,...,-14.67,-6.15,-9.06,104514.0,0.66,1.44,10.95,2.37,2025-03-10,2025-03-10 21:11:43.846563
361837,2025-02-01,9958,2025-03-10,1250268.0,778820.0,60.53,-9.99,2639353.0,2077296.0,27.06,...,-4.97,6.00,29.74,256897.0,4.87,10.27,50.75,15.59,2025-03-11,2025-03-10 21:11:43.846563
361838,2025-02-01,9960,2025-03-10,62874.0,47141.0,33.37,29.05,111595.0,96599.0,15.52,...,16.55,1.79,-1.56,33593.0,1.87,3.32,21.41,5.33,2025-03-11,2025-03-10 21:11:43.846563


### backtest

In [2]:
roe = get_factor('roe')
pbr = get_factor('股價淨值比', asc=False)
mtm = get_factor('mtm_3m')

roe_strategy = backtesting(roe>=0.99, 'QR')
pbr_strategy = backtesting(pbr>=0.95, 'QR')
mtm_strategy = backtesting(mtm>=0.95, 'QR')

,Strategy,0050.TT
Annual return,12.44%,10.83%
Total return,976.40%,733.53%
Max drawdown,-53.65%,-55.66%
Annual volatility,17.42%,19.20%
Sharpe ratio,0.71,0.56
Calmar ratio,0.23,0.19
beta,0.42,-


,Strategy,0050.TT
Annual return,16.50%,10.83%
Total return,2150.49%,733.53%
Max drawdown,-66.87%,-55.66%
Annual volatility,16.07%,19.20%
Sharpe ratio,1.0,0.56
Calmar ratio,0.25,0.19
beta,0.51,-


,Strategy,0050.TT
Annual return,15.33%,10.83%
Total return,1698.89%,733.53%
Max drawdown,-66.14%,-55.66%
Annual volatility,21.93%,19.20%
Sharpe ratio,0.7,0.56
Calmar ratio,0.23,0.19
beta,0.67,-


In [3]:
strategies = {
    'roe': (roe_strategy, 1),
    'pbr': (pbr_strategy, 1),
    'mtm': (mtm_strategy, 1),
}
multi_strategy = multi_backtesting(strategies, None)

,Meta strategy,roe,pbr,mtm,0050.TT
Annual return,15.29%,12.44%,17.18%,15.33%,10.83%
Total return,1686.11%,976.40%,2383.02%,1698.89%,733.53%
Max drawdown,-62.64%,-53.65%,-66.87%,-66.14%,-55.66%
Annual volatility,16.35%,17.42%,16.09%,21.93%,19.20%
Sharpe ratio,0.94,0.71,1.1,0.7,0.56
Calmar ratio,0.24,0.23,0.26,0.23,0.19
beta,0.55,0.42,0.51,0.67,-


In [5]:
multi_strategy.position_info

,stock_id,name,buy_date,sell_date,警示狀態,漲跌停,前次成交量_K,前次成交額_M,季報公佈日期,月營收公佈日期,板塊別,主產業別,weight
strategy,,,,,,,,,,,,,
roe,1436,華友聯,2024-11-15,2025-04-01,=,=,457.0,65.24,2024-11-04,2025-02-05,上市一般版,M2500 建材營造,0.017544
roe,1446,宏和,2024-11-15,2025-04-01,=,=,516.0,21.98,2024-11-12,2025-02-10,上市一般版,M1400 紡織纖維,0.017544
roe,1519,華城,2024-11-15,2025-04-01,=,=,1254.0,664.15,2024-11-12,2025-02-10,上市一般版,M1500 電機機械,0.017544
roe,2101,南港,2024-11-15,2025-04-01,=,=,1389.0,63.72,2024-11-13,2025-02-06,上市一般版,M2100 橡膠工業,0.017544
roe,2404,漢唐,2024-11-15,2025-04-01,=,=,2788.0,1265.70,2024-11-12,2025-02-10,上市一般版,M2300 電子工業,0.017544
...,...,...,...,...,...,...,...,...,...,...,...,...,...
mtm,8249,菱光,2024-11-15,2025-04-01,=,=,4907.0,297.48,2024-11-08,2025-02-08,上市一般版,M2300 電子工業,0.002625
mtm,8261,富鼎,2024-11-15,2025-04-01,=,=,584.0,54.12,2024-10-29,2025-02-06,上市一般版,M2300 電子工業,0.002625
mtm,8404,百和興業-KY,2024-11-15,2025-04-01,=,=,3642.0,104.60,2024-11-08,2025-02-05,上市一般版,M9900 其他,0.002625
